In [1]:
import tushare as ts
import os
import datetime
import pymongo

In [10]:
client = pymongo.MongoClient("127.0.0.1", 27017)
db = client.Common

In [11]:
stockData = ts.get_stock_basics()
stockDict = {}
industryDict = {}
for k, v in stockData.iterrows():
    name = v["name"].decode("utf-8")
    industry = v.industry.decode("utf-8")
    item = {
        "code": k, 
        "name": name,
        "industry": industry,
        "province": v.area.decode("utf-8"),
        "value": v.outstanding,
        "pe": v.pe
    }
    try:
        time = datetime.datetime.strptime(str(v.timeToMarket), "%Y%m%d")
        item["foundTime"] = time
    except:
        pass
    stockDict[k] = item
    
    if industry not in industryDict:
        industryDict[industry] = [k]
    else:
        industryDict[industry].append(k)
    
stocks = []
count = 0
for k, v in stockDict.iteritems():
    count = count + 1
    v["_id"] = count
    v["concept"] = []
    stocks.append(v)
    
    
# 概念板块
conceptData = ts.get_concept_classified()
conceptDict = {}
for k, v in conceptData.iterrows():
    code = v["code"]
    if code in stockDict:
        value = v["c_name"].decode("utf-8")
        stockDict[code]["concept"].append(value)
        if value not in conceptDict:
            conceptDict[value] = [v.code]
        else:
            conceptDict[value].append(v.code)
                

concepts = []
industries = []
for config in [[conceptDict, concepts],
               [industryDict, industries]]:
    count = 0
    for k, v in config[0].iteritems():
        count = count + 1
        config[1].append({
            "_id": count,
            "name": k,
            "stocks": sorted(v)
        })
        
print("Done")

Done


In [12]:
collection = db.Stock
collection.drop()
collection.insert_many(stocks)
collection.create_index([("code", pymongo.ASCENDING)], unique=True)
collection.create_index([("name", pymongo.ASCENDING)], unique=True)
collection.create_index([("industry", pymongo.ASCENDING)])
collection.create_index([("province", pymongo.ASCENDING)])
collection.create_index([("value", pymongo.ASCENDING)])
collection.create_index([("foundTime", pymongo.ASCENDING)])
collection = db.Stock

for config in [["Industry", industries], ["Concept", concepts]]:
    collection = db[config[0]]
    collection.drop()
    collection.insert_many(config[1])
    collection.create_index([("name", pymongo.ASCENDING)], unique=True)